In [1]:
import requests as req

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import feedparser

from selenium import webdriver

from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By

from googletrans import Translator

import time

import tqdm

import warnings
warnings.filterwarnings('ignore')

In [144]:
PATH = ChromeDriverManager().install()

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.78M/6.78M [00:03<00:00, 2.13MB/s]


In [145]:
driver = webdriver.Chrome(PATH)

In [164]:
driver.get('https://www.naturaselection.com/es/tienda/ropa')

In [147]:
#aceptar las cookies
time.sleep(2)

driver.find_element(By.XPATH, '/html/body/section/div/div/div[1]/div[2]/button').click()

In [165]:
#nos dirigimos hacia el primer producto
time.sleep(2)

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

driver.find_element(By.CLASS_NAME, 'product-card__image.u-image-switch.gtm-eh-track-product-click').click()

In [166]:
product_name = []
article_id = []
colour = []
product_type = []
detail_desc = []

contador = 0


while contador < 50:
    
    time.sleep(1)
    
    try:
    
        art = driver.find_element(By.CLASS_NAME, 'product__reference.u-light-text').text

        if art not in article_id:

            article_id.append(art)

            product_name.append(driver.find_element(By.CLASS_NAME, 'product__title.title2').text)

            tmp = []
            for e in driver.find_elements(By.CLASS_NAME, 'colors__link.js-product-color'):
                    tmp.append(e.get_attribute('data-variant-color'))

            colour.append(tmp)

            detail_desc.append(driver.find_element(By.CLASS_NAME, 'product__description-top').text.replace('\n',' '))


            tmp2 = []
            for e in driver.find_elements(By.CLASS_NAME, 'section'):
                tmp2.append(e.text)

            product_type.append(tmp2[2])
            
            contador += 1

            driver.find_element(By.CLASS_NAME, 'cn_element_products_2_image').click()
            
            print(contador)

        else:
            try:
                driver.find_element(By.CLASS_NAME, 'cn_element_products_2_image').click()
            
            except NoSuchElementFound:
                break
    
    except NoSuchElementException:
        driver.find_element(By.CLASS_NAME, 'cn_element_products_2_image').click()
    
    except Exception as e:
        break

driver.quit()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [175]:
natura = pd.DataFrame({'product_name':product_name,
                       'article_id':article_id,
                       'colour': colour,
                       'product_type': product_type,'detail_desc': detail_desc})

In [183]:
natura = natura.explode("colour")

In [184]:
natura.head()

,product_name,article_id,colour,product_type,detail_desc
0,Chaleco tejano,REF 139307,BLUE,Chalecos,"Descripción Chaleco tejano sin mangas, con bol..."
1,Pantalón Jane índigo,REF 139308,DARKBLUE,Pantalones,Descripción Pantalón de corte recto confeccion...
2,Jersey cuello pico rayas,REF 139286,BROWN,Jerséis y cárdigans,Descripción Jersey de punto con diseño de raya...
3,Bolso pequeño bandolera ante,REF 139941,BLUE,Bolsos y monederos,Descripción Bolso pequeño de ante con asa larg...
3,Bolso pequeño bandolera ante,REF 139941,CHOCOLATE,Bolsos y monederos,Descripción Bolso pequeño de ante con asa larg...


In [187]:
natura.insert(0, 'brand_name', 'NATURA')

In [189]:
natura = natura.drop(columns = 'section')

In [191]:
natura.insert(5, 'section', 'Mujer')

In [193]:
natura.to_csv('../ETL-project/natura-clean-data.csv', index = False)